# XGBoost Demo

## Download dataset

Dataset used - Titanic @ https://www.kaggle.com/competitions/titanic/data

In [6]:
!wget -nv https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv titanic.csv

2022-11-24 08:58:06 URL:https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv [60302/60302] -> "titanic.csv" [1]
wget: unable to resolve host address ‘titanic.csv’
FINISHED --2022-11-24 08:58:06--
Total wall clock time: 0.2s
Downloaded: 1 files, 59K in 0.001s (45.0 MB/s)


## Clean and transform data

In [50]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv("titanic.csv")
df = df.dropna()

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [14]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [55]:
df_features = df[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

# Binary encode gender
df_features["Sex"] = df_features["Sex"].apply(lambda x: 1 if x == "female" else 0)

# Get dummy variables for embarked
df_features = df_features.join(pd.get_dummies(df_features["Embarked"], prefix="Embarked")).drop("Embarked", axis=1)

df_features.head()

/tmp/ipykernel_8785/1641455195.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features["Sex"] = df_features["Sex"].apply(lambda x: 1 if x == "female" else 0)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
1,1,1,1,38.0,1,0,71.2833,1,0,0
3,1,1,1,35.0,1,0,53.1000,0,0,1
6,0,1,0,54.0,0,0,51.8625,0,0,1
10,1,3,1,4.0,1,1,16.7000,0,0,1
11,1,1,1,58.0,0,0,26.5500,0,0,1


## Save data to S3

In [58]:
import sagemaker
import boto3

# https://towardsdatascience.com/train-xgboost-models-in-amazon-sagemaker-in-4-simple-steps-4eb3e104ee61
# https://www.youtube.com/watch?v=A1yqn7aab6w
# https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex.html

## Train and tune model

## Deploy model